<a href="https://www.kaggle.com/code/yasstake/kaggle-baseline?scriptVersionId=124130768" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 趣旨

はじめてなのに思いつくままに始めてみたら、時間を大量に消費して行き詰ってしまった。体系的なアプローチが必要そうだ。

そこで「Kaggleで磨く機械学習の実践力（諸橋政幸　著）」のステップに従いやってみることにした。この本はTitanicを例にStepByStepでやり方を書いてあるハンズオン形式の本。ちょうど今回の例もTitanicと同じ２値問題なので応用できそう。

本の章毎にStepByStepで今回の課題をやってみる(具体的には４章から始まる)

著作権の問題もあるので章の名前をのみを引用しています。
そのため全体の考え方などは本をみないとわからないかもしれません。この本、初心者の私にとってはとてもわかりやすかったです。ぜひ、章番号とあわせて読んでみることおすすめ。

# 4 ベースラインの作成

* lightgbmとk-fold法を用いたシンプルなベースラインを作成する
* 特徴量エンジニアリングは最初はやらない。
  * 数値データは全部利用する。
  * 文字列データは一旦削除する（ベースライン完了後の特徴量エンジニアリングで実施する[予定]）

## 4.2分析設計

課題が何であるかを明確にする。
* 目的変数: カラム['target_label']の値
* 目的変数の特徴:「1＝重症」、「0＝重症ではない」の２値
* 評価指標: AUC(Area Under the Curve)

## 4.3 ファイルの読み込み

* 最初に必要ライブラリをインポートする
* 次にデータファイルを読み込む。データファイルは以下の２つ
  * `test_df.csv` テスト（提出用）データ
  * `train_df.csv`　訓練用データ

In [ ]:
! pip install numpy
! pip install sklearn
! pip install pandas
! pip install matplotlib


In [ ]:
# 必要ファイルの読み込み
import numpy as np
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# データの読み込み

# for local
#DATA_DIR = './input/'

# for kaggle notebook
DATA_DIR = '/kaggle/input/prediction-of-seriously-ill-patients/'

test_df = pd.read_csv(DATA_DIR + 'test_df.csv')
train_df = pd.read_csv(DATA_DIR + 'train_df.csv')


## 4.4 データの確認（簡易）

In [ ]:
# レコード数とカラム数の確認

print('train_df.shape: ', train_df.shape)
print('test_df.shape: ', test_df.shape)

### 4.4.2 カラムごとのデータの種類の確認

In [ ]:
# show first 5 rows
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
print(train_df.head())

In [ ]:
# show each data types and non null count
print(train_df.info())

In [ ]:
def get_object_column_name(df):
    '''
    returns object columns in dataframe
    '''
    return df.select_dtypes(include=['object']).columns

def strip_object_columns(df):
    '''
    drop columns which has object type
    '''
    return df.select_dtypes(exclude=['object'])
    


print('train_df object columns: ', get_object_column_name(train_df))
print('train_df not object columns: ', strip_object_columns(train_df).columns)


### 4.4.3 欠損値の確認

In [ ]:
train_df.isnull().sum()

# 4.5 データセットの作成

In [ ]:
x_train, y_train, id_train = train_df.drop(['id', 'target_label'], axis=1), train_df['target_label'], train_df['id']
x_test, id_test = test_df.drop(['id'], axis=1), test_df['id']

In [ ]:
print("x_train:{} | y_train: {} | id_train: {}".format(x_train.shape, y_train.shape, id_train.shape))
print("x_test:{} | id_test: {}".format(x_test.shape, id_test.shape))

# 4.6 バリデーション設計

## 4.6.2 ホールドアウト検証と交差検証

クロスバリデーション（交差検証）を採用して実装してみる。

```
`StratifiedKFold`は、クラスの分布がバランスよくなるように、データを分割する交差検証手法の一種です。具体的には、データセットをk個のサブセットに分割し、各サブセットがクラスの比率を保ったままランダムに抽出されるようにします。これにより、モデルの性能を評価するためのデータが、すべてのクラスにわたって均等に分散することが保証されます。
```

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

*Params*
 * `n_splits` 分割数
 * `shuffle` バッチ毎にシャッフルするか？（default False)
 * `random_state` クラス毎のシャッフル設定　https://scikit-learn.org/stable/modules/cross_validation.html#stratified-k-fold
 

In [ ]:
from sklearn.model_selection import StratifiedKFold

def get_fold(n_splits=5, random_state=1):
    return StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state).split(x_train, y_train)


# データがバランスしているか確認（数・割合）
for i_fold, (train_index, validate_index) in enumerate(get_fold()):
    xf_train, xf_validate = x_train.iloc[train_index], x_train.iloc[validate_index]
    yf_train, yf_validate = y_train.iloc[train_index], y_train.iloc[validate_index]
    
    print('#{}| xf_train: {} | yf_train: {} (serious: {}) | xf_validate: {} | yf_validate: {} (serious: {})'
          .format(i_fold, xf_train.shape, yf_train.shape, yf_train.mean(),
                  xf_validate.shape, yf_validate.shape, yf_validate.mean())
    )



In [ ]:
param_fixed = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'num_leaves': 2**5,
    'n_estimators': 10000,
    'random_state': 1,
    'importance_type': 'gain',
}

def train_lgbt(x_train, y_train, x_validate, y_validate, param_fixed):
    import lightgbm as lgb
    from sklearn.metrics import roc_auc_score
    from lightgbm import early_stopping
    
    param = param_fixed.copy()
    
    model = lgb.LGBMClassifier(**param)
    model.fit(strip_object_columns(x_train), y_train, eval_set=[(strip_object_columns(x_validate), y_validate)], callbacks=[early_stopping(100, verbose=False)])
    y_pred = model.predict_proba(strip_object_columns(x_validate))[:, 1]
    auc = roc_auc_score(y_validate, y_pred)
    
    return model, auc



In [ ]:
# execute cross validation

def train_fold():
    aucs = []
    models = []

    for i_fold, (train_index, validate_index) in enumerate(get_fold()):
        xf_train, xf_validate = x_train.iloc[train_index], x_train.iloc[validate_index]
        yf_train, yf_validate = y_train.iloc[train_index], y_train.iloc[validate_index]
    
        model, auc_validate = train_lgbt(xf_train, yf_train, xf_validate, yf_validate, param_fixed)
    
        print('\tFold#{}  AUC validate: {}'.format(i_fold, auc_validate))
        
        aucs.append(auc_validate)
        models.append(model)
        
    return np.array(aucs), models

aucs, models = train_fold()

## 4.7.2 クロスバリデーションの場合

説明変数の重要度の算出を行う。k個あるモデルにおける寄与度の平均をとる。

In [ ]:
def display_importance(models):
    importances = pd.DataFrame()
    for i, model in enumerate(models):
        importances['importance_{}'.format(i)] = model.feature_importances_
    importances['importance'] = importances.mean(axis=1)
    importances['feature'] = strip_object_columns(x_train).columns
    importances = importances.sort_values('importance', ascending=False)
    print(importances[['feature', 'importance']])
    
print('AUC mean: {}'.format(aucs.mean()))
display_importance(models)

## 4.8　モデル推論

k-fold法では、２つの方法がとれるようだ。
1. k-foldでトレーニングしたモデルk個のアンサンブルをとる。
2. k-foldでは、特徴量・ハイパーパラメータの有効性の確認を行い、同じパラメータで全データでトレーニングする。

ここでは１の方法で一番簡単と思われる平均値をとって推論を行う。

In [1]:
def predict(models, x_test):
    y_pred = np.zeros(len(x_test))
    for model in models:
        y_pred += model.predict_proba(strip_object_columns(x_test))[:, 1]
    y_pred /= len(models)
    return y_pred

pred = pd.DataFrame(predict(models, x_test), columns=['target_label'])
pred['id'] = id_test


def create_submission(df):
    FILE_NAME='submission.csv'
    df.to_csv(FILE_NAME, index=False, header=True)
    print('Write complete to [{}]'.format(FILE_NAME))
    
create_submission(pred)


NameError: name 'pd' is not defined